In [ ]:
from kafka import KafkaConsumer
import json
try:
        # Create KafkaConsumer
        consumer = KafkaConsumer(
            "powerlevel",
            bootstrap_servers=['telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443'],
            value_deserializer=lambda v: v,  # Keep value as bytes for JSON decoding
            group_id="sustaingroup01",
            auto_offset_reset='earliest',
            consumer_timeout_ms=20000,
            security_protocol='SSL',  # Add SSL configuration if needed
            ssl_cafile='./telemetry_zagaopenshift.pem',
            ssl_certfile='./telemetry_zagaopenshift.crt'
        )
        print("Kafka consumer created successfully.")
except Exception as e:
        print(f"Error creating Kafka consumer: {e}")

In [ ]:
messages = []
try:
        # logger.info("Starting message consumption")
    for message in consumer:
            # logger.debug(f"Consumed message: {message.value}")
            json_message = json.loads(message.value.decode('utf-8'))  # Decode and parse JSON
            messages.append(json_message)
            if len(messages) > 0:
                print("Message consumed")
                consumer.close()
except KeyboardInterrupt:
        # logger.info("Stopping consumer...")
        print("Keyboard Interrupt")
except Exception as e:
        # logger.error(f"An error occurred: {e}")
        print(e)
finally:
        print(messages)

In [ ]:
from pyiceberg.catalog import load_catalog
catalog = load_catalog(
        "docs",
    **{
            "uri": "thrift://hive-ms-trino.apps.zagaopenshift.zagaopensource.com:9888",
            "s3.endpoint": "http://minio-lb.apps.zagaopenshift.zagaopensource.com:9009",
            "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
            "s3.access-key-id": "minioAdmin",
            "s3.secret-access-key": "minio1234",
    }
)

In [ ]:
namespace = 'docs_powerleveldata'  # Replace with your namespace (database) name
tables = catalog.list_tables(namespace)

# Print the list of tables
for table in tables:
    print(table)


In [ ]:
table = catalog.load_table("docs_powerleveldata.demo")

In [ ]:
# Append the new data to the table
with table.overwrite() as writer:
    for record in messages:
        writer(record)
print(table.scan().to_arrow())

In [ ]:
table.scan().to_arrow()

In [ ]:
catalog.create_namespace("docs_powerleveldata")

In [ ]:
catalog.list_namespaces()

In [ ]:
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    LongType,
    IntegerType,
    StringType,
    NestedField,
    StructType,
    ListType
)
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import DayTransform
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform

# Define the schema for the spans
schema = Schema(
    NestedField(
        field_id=1,
        name="alldata",  # Name of the column to store all data
        field_type=StringType(),
        required=False
    )
)

table = catalog.create_table(
        identifier="docs_powerleveldata.demo2", #mendtion identifier for namespace and create a table name
        schema=schema ,
        location="s3a://mariselvam/powerleveldata2" ,#mention the  minio bucket name and mendtion path name
)


In [ ]:
from pyiceberg.io.pyarrow import PyArrowFileIO
import pyarrow as pa
import json

# Serialize each message into a JSON string
serialized_data = [json.dumps(msg) for msg in messages]

# Create a PyArrow Table with a single column containing the serialized data
df = pa.Table.from_pylist([{"alldata": data} for data in serialized_data])
table.append(df)

In [ ]:
table.scan().to_arrow()

In [1]:
from kafka import KafkaConsumer
import json
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import StringType, NestedField
from pyiceberg.io.pyarrow import PyArrowFileIO
import pyarrow as pa

def create_kafka_consumer():
    try:
        consumer = KafkaConsumer(
            "powerlevel",
            bootstrap_servers=['telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443'],
            value_deserializer=lambda v: v,  # Keep value as bytes for JSON decoding
            group_id="sustaingroup01",
            auto_offset_reset='earliest',
            consumer_timeout_ms=20000,
            security_protocol='SSL',  # Add SSL configuration if needed
            ssl_cafile='../telemetry_zagaopenshift.pem',
            ssl_certfile='../telemetry_zagaopenshift.crt'
        )
        print("Kafka consumer created successfully.")
        return consumer
    except Exception as e:
        print(f"Error creating Kafka consumer: {e}")
        return None

def consume_messages(consumer):
    messages = []
    try:
        for message in consumer:
            json_message = json.loads(message.value.decode('utf-8'))  # Decode and parse JSON
            messages.append(json_message)
            if len(messages) > 0:
                print("Message consumed")
                consumer.close()
                break
    except KeyboardInterrupt:
        print("Keyboard Interrupt")
    except Exception as e:
        print(e)
    finally:
        print(messages)
    return messages

def load_catalog_and_list_tables():
    catalog = load_catalog(
        "docs",
        **{
            "uri": "thrift://hive-ms-trino.apps.zagaopenshift.zagaopensource.com:9888",
            "s3.endpoint": "http://minio-lb.apps.zagaopenshift.zagaopensource.com:9009",
            "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
            "s3.access-key-id": "minioAdmin",
            "s3.secret-access-key": "minio1234",
        }
    )
    namespace = 'docs_powerleveldata'  # Replace with your namespace (database) name
    tables = catalog.list_tables(namespace)
    for table in tables:
        print(table)
    return catalog, tables

def create_or_load_table(catalog, tables):
    table_name = "demo5"
    full_table_name = ('docs_powerleveldata', table_name)

    if full_table_name in tables:
        print(f"Table {full_table_name} already exists.")
        table = catalog.load_table(full_table_name)
    else:
        print(f"Table {full_table_name} does not exist. Creating table...")
        schema = Schema(
            NestedField(
                field_id=1,
                name="alldata",  # Name of the column to store all data
                field_type=StringType(),
                required=False
            )
        )
        table = catalog.create_table(
            identifier=full_table_name,  # Mention identifier for namespace and create a table name
            schema=schema,
            location="s3a://mariselvam/powerlevelsampledata"  # Mention the Minio bucket name and mention path name
        )
    return table

def append_data_to_table(table, messages):
    # Serialize each message into a JSON string
    serialized_data = [json.dumps(msg) for msg in messages]

# Create a PyArrow Table with a single column containing the serialized data
    df = pa.Table.from_pylist([{"alldata": data} for data in serialized_data])
    table.append(df)
    
    # Append data to the Iceberg table
    # with table.new_append() as appender:
    #     appender.append_table(df)
    #     appender.commit()

def main():
    consumer = create_kafka_consumer()
    if not consumer:
        return
    
    messages = consume_messages(consumer)
    if not messages:
        return
    
    catalog, tables = load_catalog_and_list_tables()
    table = create_or_load_table(catalog, tables)
    append_data_to_table(table, messages)
    print("Data appended successfully.")

if __name__ == "__main__":
    main()


Kafka consumer created successfully.
Message consumed
[{'resourceMetrics': [{'resource': {'attributes': [{'key': 'service.name', 'value': {'stringValue': 'kepler'}}, {'key': 'net.host.name', 'value': {'stringValue': 'kepler.openshift-power-monitoring.svc.cluster.local'}}, {'key': 'server.address', 'value': {'stringValue': 'kepler.openshift-power-monitoring.svc.cluster.local'}}, {'key': 'service.instance.id', 'value': {'stringValue': 'kepler.openshift-power-monitoring.svc.cluster.local:9103'}}, {'key': 'net.host.port', 'value': {'stringValue': '9103'}}, {'key': 'http.scheme', 'value': {'stringValue': 'http'}}, {'key': 'server.port', 'value': {'stringValue': '9103'}}, {'key': 'url.scheme', 'value': {'stringValue': 'http'}}]}, 'scopeMetrics': [{'scope': {'name': 'otelcol/prometheusreceiver', 'version': '0.103.0'}, 'metrics': [{'name': 'kepler_container_bpf_net_tx_irq_total', 'description': 'Aggregated value in bpf_net_tx_irq value from bpf', 'sum': {'dataPoints': [{'attributes': [{'key': 